In [33]:
import pandas as pd
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import * 
from sklearn import preprocessing

In [5]:
model = load_model('clothing-classifier-model.h5')

In [9]:
label_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

In [11]:
# Get test data
test_df = pd.read_csv('fashion-mnist_test.csv')
test_df.loc[:, 'label'] = test_df.loc[:, 'label'].map(label_mapping)
x_test = test_df.iloc[:,1:]
y_test = test_df.loc[:,'label']


#Test
encoder = preprocessing.OneHotEncoder(sparse_output=False)
y_test = encoder.fit_transform(test_df.loc[:,'label'].values.reshape(-1,1)) # OneHot encoding
feature_names = [name.split('_', 1)[1] for name in encoder.get_feature_names_out()]

x_test/=255 #Normalize
y_test = pd.DataFrame(y_test, columns=feature_names)

In [13]:
## Predicting with Model #
y_pred_probs = model.predict(x_test)

indices = np.argmax(y_pred_probs, axis=1) #Converting probabilities

#Creating OneHot-Encoded prediction array
y_pred = np.zeros_like(y_test)
for i, index in enumerate(indices):
    y_pred[i,index] = 1
y_pred = pd.DataFrame(y_pred,columns=y_test.columns)

313/313 [==============================] - 0s 1ms/step


In [30]:
## Metrics ##

columns = ['MSE','Accuracy','Precision', 'Recall', 'F1']
metrics_list = []

for col in y_test.columns:
    feat_pred = y_pred.loc[:,col]
    feat_test = y_test.loc[:,col]
    MSE = mean_squared_error(feat_pred, feat_test)
    acc = accuracy_score(feat_pred, feat_test).round(3)
    prec = precision_score(feat_pred, feat_test).round(3)
    recall = recall_score(feat_pred, feat_test).round(3)
    F1 = f1_score(feat_pred,feat_test).round(3)
    
    metrics_list.append([MSE, acc, prec, recall, F1])
    
metrics_df = pd.DataFrame(metrics_list, columns=columns, index=y_test.columns)

In [32]:
metrics_df.to_csv('metrics.csv', index=True)